In [1]:
import ee
ee.Initialize()

In [2]:
import geepy
import math
import sys

In [3]:
config = geepy.params.configParams('input_classification_v31.json')

amostras = ee.FeatureCollection(config.params['samples']['2016v13'])

watersheds = ee.FeatureCollection(config.params['studyArea']) 

tiles = ee.FeatureCollection(config.params['lsTiles'])

pivots = {i: ee.FeatureCollection(config.params['pivots'] + i) for (i) in config.params['years2process']}

bands = config.params['bandParams']

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['sr']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['sr']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['sr']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [4]:
tile = [geepy.feature.getTiles(tiles, tile, 'TILE_T') for tile in config.params['grid']['tiles2process']]

In [5]:
# Modelo digital de elevação
srtm = ee.Image(config.params['srtm'])
#alos = ee.Image(config.params['alos']).select('MED')

# Localities distance
towns = ee.FeatureCollection(config.params['towns'])
rivers = ee.FeatureCollection(config.params['hidroBDGEx'])

dtown = towns.distance(config.params['radist'])
driver = rivers.distance(config.params['radist'])

slope = ee.Terrain.slope(srtm)
aspect = ee.Terrain.aspect(srtm).divide(180).multiply(math.pi).sin()
hillshade = ee.Terrain.hillshade(srtm)

#slope = ee.Terrain.slope(alos)
#aspect = ee.Terrain.aspect(alos).divide(180).multiply(math.pi).sin()
#hillshade = ee.Terrain.hillshade(alos)

ntl30m = {}
for i in config.params['years2process']:
    viirs = ee.Image(config.params['VIIRS'][i]).select('avg_rad').divide(100)
    ntl30m[i] = viirs.resample('bilinear').reproject(
        crs = viirs.projection().crs(),
        scale = 30
    )

In [6]:
#### Test water mask
wmask = driver.lt(300)

In [7]:
def normalizeBands(img, img_min, img_max):
    """
        normalizeBands help
        
        (img - min)/(max - min)
    """
    
    numer = img.subtract(img_min)
    denon = img_max.subtract(img_min)
    
    normalized = numer.divide(denon)
    
    return normalized

In [8]:
landsat = {}
correctedImages = {}

for year in config.params['years2process']:
    landsat[year] = ee.Image('users/fernandompimenta/AIBA/landsat/landsat'+year+'v31')
    
landsat_min = ee.Image('users/fernandompimenta/AIBA/landsat/landsat19902018v31Min')
landsat_max = ee.Image('users/fernandompimenta/AIBA/landsat/landsat19902018v31Max')

In [9]:
normalized = {}
for year in config.params['years2process']:
    normalized[year] = normalizeBands(landsat[year], landsat_min, landsat_max)

In [10]:
tasks = {year: geepy.image.send2asset(normalized[year], config.params['extent'], 'normalized'+year+'v31', 'users/fernandompimenta/AIBA/normalized/normalized'+year+'v31', 30) for year in config.params['years2process']}

In [11]:
for i in tasks.keys():
  [tasks[i].start()]

In [19]:
for year in config.params['years2process']:
    if(int(year) < 2002):
        satellite = 'l5'
    elif(int(year) in (2002, 2011, 2012)):
        satellite = 'l7'
    elif(int(year) > 2002 and int(year) < 2011):
        satellite = 'l5'
    elif(int(year) == 2018):
        satellite = 'l8'
    else:
        satellite = 'l8'
        
    normalized[year] = geepy.image.img2Band(normalized[year], ntl30m[year], 'ntl')
    normalized[year] = geepy.image.img2Band(normalized[year], wmask, 'wmask')
    normalized[year] = geepy.image.img2Band(normalized[year], srtm, 'srtm')
    normalized[year] = geepy.image.img2Band(normalized[year], slope, 'slope')
    normalized[year] = geepy.image.img2Band(normalized[year], aspect, 'aspect')
    normalized[year] = geepy.image.img2Band(normalized[year], hillshade, 'hillshade')
    normalized[year] = geepy.image.img2Band(normalized[year], dtown, 'dtown')
    normalized[year] = geepy.image.img2Band(normalized[year], driver, 'driver')
    
    normalized[year] = geepy.image.calcNDBI(normalized[year])
    normalized[year] = geepy.image.calcNDVI(normalized[year])
    normalized[year] = geepy.image.calcEVI(normalized[year])
    normalized[year] = geepy.image.calcSAVI(normalized[year])
    normalized[year] = geepy.image.calcNDWI_L(normalized[year])
    normalized[year] = geepy.image.calcNDWI_WB(normalized[year])
    normalized[year] = geepy.image.calcRatio(normalized[year])
    
    normalized[year] = geepy.image.tassCapTransformation(normalized[year], satellite)

    ndvithermal = normalized[year].select('ndvi').divide(normalized[year].select('thermal'))
    normalized[year] = geepy.image.img2Band(normalized[year], ndvithermal, 'ndvithermal')

In [10]:
# mapa = geepy.maps.geeMap()

# ano = '2018'

# #for i in config.params['years2process']:
# mapa.addLayer(normalized[ano], name=ano)
# mapa.addControls()
# mapa.show()

In [12]:
# landsat = {}

# correctedImages = {}

# for year in config.params['years2process']:  
#     start_d = year + config.params['period']['dry']['start']
#     end_d = year + config.params['period']['dry']['end']


#     #print("Start-End dry season %s %s" %(start_d, end_d))

#     if(int(year) < 2002):
#         filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
#         satellite = 'l5'
#     elif(int(year) in (2002, 2011, 2012)):
#         filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
#         satellite = 'l7'
#     elif(int(year) > 2002 and int(year) < 2011):
#         filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
#         satellite = 'l5'
#     elif(int(year) == 2018):
#         filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, '2018-09-06').map(geepy.image.maskLandsatSR)
#         satellite = 'l8'
#     else:
#         filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
#         satellite = 'l8'
  

#     correctedImages[year] = filtered.map(geepy.image.edgeRemoval).median()
    
# #     sys.stdout.write("\rProcessing Landsat data: %s" % year)
# #     sys.stdout.flush()
    
# collectionFromImages = ee.ImageCollection.fromImages(list(correctedImages.values()));

# images_min = collectionFromImages.min()
# images_max = collectionFromImages.max()

# for year in config.params['years2process']:
#     normalized = normalizeBands(correctedImages[year], images_min, images_max)
    
#     normalized = geepy.image.img2Band(normalized, ntl30m[year], 'ntl')
#     normalized = geepy.image.img2Band(normalized, wmask, 'wmask')
#     normalized = geepy.image.img2Band(normalized, srtm, 'srtm')
#     normalized = geepy.image.img2Band(normalized, slope, 'slope')
#     normalized = geepy.image.img2Band(normalized, aspect, 'aspect')
#     normalized = geepy.image.img2Band(normalized, hillshade, 'hillshade')
#     normalized = geepy.image.img2Band(normalized, dtown, 'dtown')
#     normalized = geepy.image.img2Band(normalized, driver, 'driver')
    
#     normalized = geepy.image.calcNDBI(normalized)
#     normalized = geepy.image.calcNDVI(normalized)
#     normalized = geepy.image.calcEVI(normalized)
#     normalized = geepy.image.calcSAVI(normalized)
#     normalized = geepy.image.calcNDWI_L(normalized)
#     normalized = geepy.image.calcNDWI_WB(normalized)
#     normalized = geepy.image.calcRatio(normalized)
    
#     normalized = geepy.image.tassCapTransformation(normalized, satellite)

#     ndvithermal = normalized.select('ndvi').divide(normalized.select('thermal'))
#     normalized = geepy.image.img2Band(normalized, ndvithermal, 'ndvithermal')
    
#     fEdgeRemoved_min = filtered.map(geepy.image.edgeRemoval).min()
#     fEdgeRemoved_max = filtered.map(geepy.image.edgeRemoval).max()
    
#     landsat[year] = normalized.clip(tile[0][1])
    
# #    landsat_min[year] = fEdgeRemoved_min.clip(watersheds)
# #    landsat_max[year] = fEdgeRemoved_max.clip(watersheds)
    
#     sys.stdout.write("\rProcessing Landsat data: %s" % year)
#     sys.stdout.flush()

In [13]:
#tasks = {year: geepy.image.send2asset(correctedImages[year], config.params['extent'], 'landsat'+year+'v31', 'users/fernandompimenta/AIBA/landsat/landsat'+year+'v31', 30) for year in config.params['years2process']}

In [14]:
#for i in tasks.keys():
#   [tasks[i].start()]

In [15]:
#help(geepy.image.send2asset)

In [16]:
# mapa = geepy.maps.geeMap(tile[0][1], zoom=10)

# ano = '2017'

# #for i in config.params['years2process']:
# mapa.addLayer(landsat[ano], name=ano)
# mapa.addControls()
# mapa.show()

In [20]:
landsat = normalized

In [21]:
amostragem = geepy.feature.vec2rast(amostras, 'CLASS')

n = 4
classBand = 'CLASS'
cv = [1,2,3,4,5,6,7,8,9,10,11,12]
cp = [n for i in range(len(cv))]

#cp[6] = 0   # Agricultura irrigada
cp[3] = 3000 # Formações campestres
#cp[5] = 2000 # Agricultura de sequeiro
#cp[7] = 1000 # Pastagem
cp[9] = 200  # Área urbana

samples = geepy.image.randomSamples(amostras, amostragem, n, 369, classBand, cv, cp)
# 1 Formações florestais --------------------------------- #004000
# 2 Formações savânicas ---------------------------------- #77a605
# 3 Mata ciliar ------------------------------------------ #004000
# 4 Formações campestres --------------------------------- #b8af4f
# 5 Agricultura ou pastagem ------------------------------ #f6e6db
# 6 Agricultura de sequeiro ------------------------------ #ffcaff
# 7 Agricultura Irrigada --------------------------------- #ff42f9
# 8 Pastagem  -------------------------------------------- #f4f286
# 9 Corpos d'água ---------------------------------------- #0000ff
# 10 Área urbana/Construções rurais ---------------------- #ff0000
# 11 Solo exposto ---------------------------------------- #77a605
# 12 Rochas ---------------------------------------------- #77a605

In [22]:
for year in config.params['years2process']:
    pivots[year] = geepy.feature.vec2rast(pivots[year], 'CLASS').reproject(
        crs = landsat[year].select('nir').projection().crs(),
        scale = 30
    )
    sys.stdout.write("\rProcessing Central Pivots data: %s" % year)
    sys.stdout.flush()

AttributeError: 'Image' object has no attribute 'filter'

In [23]:
# muda o valor dos pivos de 4 para 7
for year in config.params['years2process']:
    pivots[year] = pivots[year].remap([4],[7]).rename(['CLASS'])
    sys.stdout.write("\rProcessing Central Pivots data: %s" % year)
    sys.stdout.flush()

Processing Central Pivots data: 2018

In [24]:
training = samples.filter(ee.Filter.gt('randCol', 0.2))
validation = samples.filter(ee.Filter.lt('randCol', 0.2))

trained = geepy.image.trainingSamples(landsat['2016'], training)

In [25]:
# Randon Forest
classification = {year: geepy.image.randomForest(landsat[year], trained, bands, ntrees=1) for (year) in config.params['years2process']}
classRemapped = {year: classification[year][0].remap([1,2,3,4,5,6,7,8,9,10,11,12],[1,2,3,4,5,6,6,8,9,10,11,12]).rename('classification'+year) for (year) in config.params['years2process']}
finalClassification = {year: classRemapped[year].where(pivots[year].select('CLASS'), 7) for year in config.params['years2process']}

In [26]:
mapa = geepy.maps.geeMap(tile[0][1], zoom=10)

ano = '1990'

#for i in config.params['years2process']:
mapa.addLayer(finalClassification[ano], viz_params=config.params['vizParams']['classification'], name=ano)
mapa.addControls()
mapa.show()

In [20]:
# To drive
tasks = {year: geepy.image.send2drive(finalClassification[year], config.params['extent'], 'classification'+year+'v31', 'classification_v31_rf100dt', 30) for year in config.params['years2process']}

In [21]:
for i in tasks.keys():
   [tasks[i].start()]